In [1]:
using PyCall
using Distances
using StatsBase
using LinearAlgebra
using JuMP
using Gurobi
using CSV
using Distances
using DataFrames
# using PyPlot
using SparseArrays
using Printf

┌ Info: Recompiling stale cache file /Users/kamalsdu/.julia/compiled/v1.0/PyCall/GkzkC.ji for PyCall [438e738f-606a-5dbb-bf0a-cddfbfd45ab0]
└ @ Base loading.jl:1190
┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1192
┌ Info: Precompiling Gurobi [2e9cd046-0924-5485-92f1-d5272153d98b]
└ @ Base loading.jl:1192
┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1192


In [2]:
py"""
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import sys
import cv2
import os

class Triangulation:
    def load_imgs(self, path1, path2):
        self.img1 = cv2.imread(path1, cv2.CV_8UC3)
        self.img2 = cv2.imread(path2, cv2.CV_8UC3)
        # base = "/".join(path1.split("\\")[:-1])
        # cv2.imwrite(os.path.join(base, "left_loaded.png"), self.img1)
        # cv2.imwrite(os.path.join(base, "right_loaded.png"), self.img2)

    def findRootSIFTFeatures(self, n_components = None):
        class RootSIFT:
            def __init__(self):
                self.extractor = cv2.xfeatures2d.SIFT_create(n_components) if n_components != None else cv2.xfeatures2d.SIFT_create()
            def compute(self, image, kps, eps=1e-7):
                (kps, descs) = self.extractor.compute(image, kps)
                if len(kps) == 0:
                    return ([], None)

                descs /= (descs.sum(axis=1, keepdims=True) + eps)
                descs = np.sqrt(descs)
                return (kps, descs)

        class InnerFeatures:
            def __init__(self, kps, des, pos):
                self.kps = kps
                self.des = des
                self.pos = pos

        def innerRootSIFT(img):
            sift = cv2.xfeatures2d.SIFT_create(n_components) if n_components != None else cv2.xfeatures2d.SIFT_create()
            (kps, descs) = sift.detectAndCompute(img, None)

            rs = RootSIFT()
            (kps, descs) = rs.compute(img, kps)
            pos = [np.array([x.pt[0], x.pt[1]]) for x in kps]

            return kps, descs, pos

        kps1, desc1, pos1 = innerRootSIFT(self.img1)
        kps2, desc2, pos2 = innerRootSIFT(self.img2)
        self.feature_1 = InnerFeatures(kps1, desc1, pos1)
        self.feature_2 = InnerFeatures(kps2, desc2, pos2)
    def drawMatches(self, path):
        self.outImage = cv2.drawMatches(self.img1, self.feature_1.kps, self.img2, self.feature_2.kps, self.matches,outImg=None)
        cv2.imwrite(path, self.outImage)
scene = Triangulation()
"""

img1_path = "../data/dense/0000-small-left.png"
img2_path = "../data/dense/0001-small-right.png"

py"scene.load_imgs"(img1_path, img2_path)
py"scene.findRootSIFTFeatures"()

pts1 = py"scene.feature_1.pos"
pts2 = py"scene.feature_2.pos";

P_points = hcat(pts1...)'
Q_points = hcat(pts2...)'

println("size P points", size(P_points))
println("size Q points", size(Q_points))

size P points(599, 2)
size Q points(676, 2)


In [3]:
cost = pairwise(Euclidean(), P_points, Q_points; dims=1)
println(size(cost))
P = ones(size(P_points,1))
Q = ones(size(Q_points,1));

(599, 676)


In [4]:
solCount = 10
# m = JuMP.direct_model(Gurobi.Optimizer(PoolSearchMode=2, PoolSolutions=solCount, SolutionNumber=0,PoolGap = 0.001))
m = JuMP.direct_model(Gurobi.Optimizer(PoolSearchMode=2, PoolSolutions=solCount, SolutionNumber=0))

@variable(m, X[axes(cost,1), axes(cost,2)] ≥ 0, Int)
@objective(m, Min, cost ⋅ X)
@constraint(m,sum(X) .== min(sum(P), sum(Q)))
@constraint(m, X * ones(Int, length(Q)) .<= P)
@constraint(m, X'ones(Int, length(P)) .<= Q);
optimize!(m)
solution_pool = zeros(solCount, length(P),length(Q))
cnt = 0
obj = objective_value(m)

for i in 0:(solCount-1)
    try
        setparam!(m.moi_backend.inner,"SolutionNumber", i)
        xn = Gurobi.get_dblattrarray(m.moi_backend.inner, "Xn", 1, length(X))
        xn_val = Gurobi.get_dblattr(m.moi_backend.inner, "PoolObjVal")
        if(round(xn_val,digits=1) != round(obj, digits=1))
            println(cnt , " solution(s) selected")
            break
        end
        default = zeros(length(P),length(Q))
        for i in 0:length(P)-1
            default[i+1,:] = xn[(i*length(Q))+1:(i+1)*length(Q)]
        end
        solution_pool[i+1,:,:] = default
        cnt+=1
    catch 
        break
    end
end
sol_pool = deepcopy(solution_pool[1:cnt,:,:]);

Optimize a model with 1276 rows, 404924 columns and 1214772 nonzeros
Variable types: 0 continuous, 404924 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+02]
Presolve time: 2.11s
Presolved: 1276 rows, 404924 columns, 1214772 nonzeros
Variable types: 0 continuous, 404924 integer (404924 binary)

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0          0     infinity      0.0000000e+00      4s
       1       2451   4.2007444e+07   1.2475434e+03      4s
       2       6608   2.3008455e+07   1.9182428e+03      4s
       3      10781   9.0096278e+06   2.1843818e+03      5s
       4      15234   1.0535040e+04   3.7318834e+03      5s
       5      17604   1.0351183e+04   8.6437919e+03      5s

Sifting complete


Root relaxation: objective 1.032155e+04, 19225 iterations, 0.73 seconds

    N

In [5]:
solOther = sparse(sol_pool[1,:,:])
sizeOf = min(size(P,1), size(Q,1))
matched_pts1 = zeros(sizeOf,2)
matched_pts2 = zeros(sizeOf,2)
i = 1
py"""
arr = []
"""
for (x,y,v) in zip(findnz(solOther)...)
    x_pos = [P_points'[:,x][1], Q_points'[:,y][1]]
    y_pos = [P_points'[:,x][2], Q_points'[:,y][2]]
    
    # dmatch creating
    queryId = x-1
    trainId = y-1
    distance = cost[x,y]
    if(distance <= 10)
        dmatch = py"cv2.DMatch($(queryId), $(trainId),$(distance))"
        py"arr.append"(dmatch)
        matched_pts1[i,:] = [floor(x_pos[1]) floor(y_pos[1])]
        matched_pts2[i,:] = [floor(x_pos[2]) floor(y_pos[2])]
        i+=1
    end
end
py"""
scene.matches = arr
"""


# path = "../data\\pair\\lastLPMatched.png"
# py"scene.drawMatches"(path)


matched_final_1 = deepcopy(matched_pts1[1:i-1, :])
matched_final_2 = deepcopy(matched_pts2[1:i-1, :]);

In [6]:
df = DataFrame()
df.PX = matched_final_1[:,1]
df.PY = matched_final_1[:,2]
df.QX = matched_final_2[:,1]
df.QY = matched_final_2[:,2];
df

,PX,PY,QX,QY
,Float64,Float64,Float64,Float64
1,19.0,60.0,16.0,53.0
2,36.0,58.0,36.0,55.0
3,49.0,59.0,39.0,59.0
4,41.0,8.0,39.0,9.0
5,53.0,49.0,51.0,51.0
6,52.0,58.0,54.0,62.0
7,56.0,262.0,64.0,264.0
8,84.0,237.0,78.0,234.0
9,82.0,279.0,79.0,277.0


In [7]:
CSV.write("../data/dense/matchedPoints.csv",  df, writeheader=false)

"../data/dense/matchedPoints.csv"